In [ ]:
import numpy as np
import pandas as pd
import fastf1
from src.utils import get_acc_df, compare_car_speeds
from src.plotset import setup_plot

from fastf1 import plotting
import matplotlib.pyplot as plt
import seaborn as sns

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2024,'Dutch','Q')
session.load()

In [ ]:
low = [9,10,11,12]
medium = [1,2,3,4,8,13]
high = [5,6,7,14]

In [ ]:
drivers = [session.get_driver(i).Abbreviation for i in session.drivers]
drivers.remove('ALB')
drivers.remove('SAR')

In [ ]:
cars_perf =compare_car_speeds(session=session,drivers=drivers,corner_inputs={'low':low,'medium':medium,'high':high})

In [ ]:
cars_perf

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
rel_cars_perf = pd.DataFrame(mms.fit_transform(cars_perf),index=cars_perf.index,columns=cars_perf.columns)

In [ ]:
rel_cars_perf

In [ ]:
session = fastf1.get_session(2024,'Dutch','R')
session.load()

In [ ]:
get_acc_df(session=session)

In [ ]:
df1 = session.laps.pick_drivers(['NOR']).pick_quicklaps()[['Driver','LapTime','LapNumber','Stint','Compound']]
df1['LapTime'] = df1['LapTime'].dt.total_seconds()
df1[df1.Compound == 'MEDIUM'].LapTime.mean(), df1[df1.Compound == 'HARD'].LapTime.mean()

In [ ]:
df2 = session.laps.pick_drivers(['VER']).pick_quicklaps()[['Driver','LapTime','LapNumber','Stint','Compound']]
df2['LapTime'] = df2['LapTime'].dt.total_seconds()
df2[df2.Compound == 'MEDIUM'].LapTime.mean(), df2[df2.Compound == 'HARD'].LapTime.mean()

In [ ]:
df1[df1.Compound == 'MEDIUM'].LapTime.mean() - df2[df2.Compound == 'MEDIUM'].LapTime.mean()

In [ ]:
df1[df1.Compound == 'HARD'].LapTime.mean() - df2[df2.Compound == 'HARD'].LapTime.mean()

In [ ]:
setup_plot(xyticksize=24,axeslabel=30)

fig, ax = plt.subplots(figsize=(15,7))

# ax.set_axis_off()

sns.violinplot(df1,x='Driver',y='LapTime',hue='Compound',palette=['#ff8000','#ff8000'],
               inner=None,linecolor='w',order=['NOR','VER'],legend=False)
sns.stripplot(df1,x='Driver',y='LapTime',hue='Compound',palette=plotting.get_compound_mapping(session)
              ,linewidth=1,size=15,edgecolor='k',dodge=True,legend=False)

sns.violinplot(df2,x='Driver',y='LapTime',hue='Compound',palette=['#0600ef','#0600ef'],
               inner=None,linecolor='w',order=['NOR','VER'],legend=False)
sns.stripplot(df2,x='Driver',y='LapTime',hue='Compound',palette=plotting.get_compound_mapping(session)
              ,linewidth=1,size=15,edgecolor='k',dodge=True,legend=False)


ax.set_ylim(73,78)
ax.set_ylabel("Lap Time (sec)")
ax.set_xlabel("")

ax.grid(visible=False)

In [ ]:
laps = session.laps
drivers = session.drivers
drivers = [session.get_driver(i).Abbreviation for i in session.drivers]

In [ ]:
stints = laps[["Driver", "Stint", "Compound", "LapNumber"]]
stints = stints.groupby(["Driver", "Stint", "Compound"])
stints = stints.count().reset_index()
stints = stints.rename(columns={"LapNumber": "StintLength"})

In [ ]:
fig, ax = plt.subplots(figsize=(8, 9))

for driver in drivers:
    driver_stints = stints.loc[stints["Driver"] == driver]

    previous_stint_end = 0
    for idx, row in driver_stints.iterrows():
        # each row contains the compound name and stint length
        # we can use these information to draw horizontal bars
        compound_color = fastf1.plotting.get_compound_color(row["Compound"],
                                                            session=session)
        plt.barh(
            y=driver,
            width=row["StintLength"],
            left=previous_stint_end,
            color=compound_color,
            edgecolor="black",
            fill=True
        )

        previous_stint_end += row["StintLength"]


# plt.title("2024 Dutch Grand Prix Tire Strategies")
plt.xlabel("Lap Number")
plt.grid(False)
# invert the y-axis so drivers that finish higher are closer to the top
ax.invert_yaxis()

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.tight_layout()
plt.show()